In [ ]:
"""
PV Lighthouse Uncertainty Tool
Helper file
Contains many functions used for connecting to the uncertainty API and analysing its output
"""

# Define constants for plotting
FIGURE_FONT_SIZE = 16           # Font size for figure axes, titles will be (FIGURE_FONT_SIZE + 2).
TRANSPARENT_PLOTS = True        # False = white background, True = transparent (best for adding into presentations)
HISTOGRAM_COLOUR = '#d2226a'    # Hex code for the colour used in column plots
GRID_COLOUR = 'LightGrey'       # Colour of gridlines in plots
GRID_WIDTH = 0.1                # Thickness of gridlines in plots

# Import P90Client and related tools
from pvl_p90_client.client.p90_client import AuthenticationError, ServiceError, P90Client, P90ConnectionError
from pvl_p90_client.grpcclient import uncertaintyMessages_pb2, weatherData_pb2
from pvl_p90_client.grpcclient.uncertaintyMessages_pb2 import DistributionInput
from pvl_p90_client.helpers import pvl_login
from pvl_p90_client.helpers.exception import TimeStepDataError
from pvl_p90_client.helpers.request_helpers import (
    build_gaussian_distribution,
    build_skewed_gaussian_distribution,
    build_distribution,
    build_module_info,
    build_result_options,
    build_simulation_options,
    build_system_info,
    build_weibull_distribution,
    build_arbitrary_distribution,
    load_weather_data_from_pvw_file,
    load_time_step_data_from_csv,
    write_timestep_data_to_csv
)


# Import other Python libraries
import logging
import datetime
import re
import time
import math
import base64
import numpy as np
import pandas as pd
from scipy.stats import norm, skewnorm
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

try:
    # # Enable the plotly renderer for Colab for interactive plots
    from google.colab import output
    # pio.renderers.default = 'colab'
    output.enable_custom_widget_manager()       # Needed to make plotly interactive plots work correctly in Colab
except:
    # Not using Google Colab
    None
try:
    from openpyxl.workbook import Workbook
except:
    # Install openpyxl
    !pip install openpyxl

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")


#### Define helper functions ####

def build_request(
    time_step_data: list[uncertaintyMessages_pb2.TimeStepData],
    distributions: list[uncertaintyMessages_pb2.Distribution] | None = None,
    number_of_years: int = 5,
    number_of_simulations: int = 5000,
    module_info: uncertaintyMessages_pb2.ModuleInfo | None = None,
    system_info: uncertaintyMessages_pb2.SystemInfo | None = None,
    p_values: list[int] = [50, 90, 95],
    p_min: float = 0.75,
    p_delta: float = 0.01
) -> uncertaintyMessages_pb2.UncertaintyRequest:
    request = uncertaintyMessages_pb2.UncertaintyRequest()

    # Module configuration
    # module_info = build_module_info(length=2.0, width=1.0, azimuth_in_rad=math.pi / 2.0)
    if module_info is None:
        # Default
        module_info = build_module_info(length=2.0, width=1.0)
    request.Module.CopyFrom(module_info)

    if system_info is None:
        # Default
        system_info = build_system_info(
            modules_per_string=1,
            inverter_efficiency_rate=0.98,
            num_strings=1,
            row_pitch_in_m=5.6,
            module_to_module_mismatch_rate=0.99,
            wiring_loss_rate=0.99,
            max_power_tracking_rate=1.0, azimuth_in_rad=math.pi / 2.0
        )
    request.System.CopyFrom(system_info)

    if number_of_years*number_of_simulations > 125000:
        raise ValueError(f"number_of_simulations*number_of_years must be less than 125000, currently {number_of_years*number_of_simulations}")

    simulation_options = build_simulation_options(number_of_years, number_of_simulations)
    request.SimulationOptions.CopyFrom(simulation_options)

    result_options = build_result_options(p_min=p_min, p_delta=p_delta, p_values=p_values)
    request.ResultOptions.CopyFrom(result_options)

    # Add modifiers if provided
    if distributions is not None:
        request.Distributions.extend(distributions)

    # Add weather data
    request.TMYDataSet.TimeStepDataPoints.extend(time_step_data)

    return request

def create_distribution(input, simToSim=None, yearToYear=None, stepToStep=None) -> uncertaintyMessages_pb2.Distribution:
    """Creates an input distribution with user-input values.
        input: the integer corresponding to the system variable which is being modified (0-16)
            GHI: 0, DiffuseFraction: 1, WindSpeed: 2, Temperature: 3, etaSTC: 4, FrontSpectral: 5, FrontSoiling: 6, RearSoiling: 7, Uc: 8,
            Uv: 9, Alpha: 10, Degradation: 11, Availability: 12, ExtraYearly: 13, HayModel: 14, UndulatingGround: 15, OpticalTracking: 16
        {X}_pdf: the probability distribution function (pdf) for {X} uncertainty (simulation, yearly, or time_step)
                    pdf format: ['FunctionName', *parameters], pdfs: 'Gaussian', 'SkewedGaussian', 'Weibull', or 'Arbitrary'
        simToSim: the pdf for simulation error (e.g. ['Gaussian', 0.01, 0.1] ; x0 = 0.01, σ = 0.1)
        yearToYear: the pdf for yearly error (e.g. ['Weibull', 0.0, 0.025, 1.7] ; x0 = 0, λ = 0.025, k = 1.7)
        time_pdf: the pdf for time step error (e.g. ['SkewedGaussian', -2.0, 1.0, 3.2] ; ɑ = -2, ζ = 1, ⍵ = 3.2)
        If any pdf's are None then they are not used.

        TODO:
            - Need to have keyword params in pdf, as the order can be random.
            - Check arbitrary works and document it.
    """
    if not input in range(17):
        raise ValueError(f"Invalid simulation input: {input}")
    input_distribution = build_distribution(
        input=input,
        sim_to_sim_distribution=simToSim and build_pdf_from_params(simToSim[0], *simToSim[1:]),   # 'and' ensures {X}_pdf=None is passed as None
        yr_to_yr_distribution=yearToYear and build_pdf_from_params(yearToYear[0], *yearToYear[1:]),
        step_to_step_distribution=stepToStep and build_pdf_from_params(stepToStep[0], *stepToStep[1:])
    )
    return input_distribution

def build_pdf_from_params(pdf_name, *params):
    # Simple method of generically sending pdf build requests
    if pdf_name == 'Gaussian':
        # build_gaussian_distribution( x0: float, sigma: float, fmax: float | None = None, num_points_in_probability_function: int | None = None, lower_limit: float | None = None, upper_limit: float | None = None, )
        return build_gaussian_distribution(*params)
    elif pdf_name == 'SkewedGaussian':
        # build_skewed_gaussian_distribution( alpha: float, zeta: float, omega: float, fmax: float | None = None, num_points_in_probability_function: int | None = None, )
        return build_skewed_gaussian_distribution(*params)
    elif pdf_name == 'Weibull':
        # build_weibull_distribution(x0: float, lambda_term: float, k: float, has_positive_polarity: bool | None = None)
        return build_weibull_distribution(*params)
    elif pdf_name == 'Arbitrary':
        # build_arbitrary_distribution(x: list[float], y: list[float])
        return build_arbitrary_distribution(*params)
    else:
        raise ValueError(f"Invalid pdf name: {pdf_name}")

def request_analysis(p90_request):
    with P90Client(version="latest-dev") as client:
        # User will be prompted to enter credentials
        credentials = pvl_login.login()
        # Send request and get results
        summary = perform_analysis(client, p90_request, credentials)
        return summary

def perform_analysis(uncertainty_client, uncertainty_request, call_credentials, show_errors=True):
    """Perform the uncertainty analysis and display results."""
    try:
        print("Starting uncertainty analysis...")
        t_start = time.time()    # get the start time
        summary: UncertaintySummary | None = uncertainty_client.send_request( #uncertainty_client,
            uncertainty_request, call_credentials, timeout=60*14
        )  # timeout in seconds
        t_request = time.time()-t_start    # get the request time in seconds
        n_total = uncertainty_request.SimulationOptions.NumberOfSimulations*uncertainty_request.SimulationOptions.NumberOfYears
        print(f"Request took {round(t_request)} s, running {n_total} simulations in total ({round(n_total/t_request)} sims/s)")
        # Display results
        if summary:
            return summary
        else:
            print("⚠️  No results received from analysis")
    except AuthenticationError as e:
        if show_errors:
            print(f"❌ Authentication Error: {e}")
    except P90ConnectionError as e:
        if show_errors:
            print(f"❌ Connection Error: {e}")
    except ServiceError as e:
        if show_errors:
            print(f"❌ Service Error: {e}")
    except TimeoutError as e:
        if show_errors:
            print(f"❌ Timeout Error: {e}")
    except ValueError as e:
        if show_errors:
            print(f"❌ Invalid Request: {e}")
    except (RuntimeError, ConnectionError) as e:
        if show_errors:
            print(f"❌ Unexpected Error: {e}")

# Main function demonstrating uncertainty analysis with proper error handling.
def main():
    try:
        # Initialize client with context manager for automatic cleanup
        with P90Client(version="latest-dev") as p90_client:
            print("Connecting to PV Lighthouse...")

            # Get authentication credentials
            print("Authenticating...")
            call_credentials = pvl_login.login()
            print("Authentication successful!")

            # Load time step data
            print("Loading time step data...")
            if (weather_data_sydney := load_weather_data_from_pvw_file("Data/sydney.pvw")) is None:
                raise TimeStepDataError("Failed to load weather data from file")
            print(f"Loaded {len(weather_data_sydney)} weather data points")

            # # Send a null request to check user claims
            # null_request = build_request(weather_data_sydney, None, number_of_years=0, number_of_simulations=0, module_info=build_module_info(length=2.0, width=1.0),
            #     system_info=build_system_info(
            #                 modules_per_string=1, inverter_efficiency_rate=0.98,
            #                 num_strings=1, row_pitch_in_m=5.6, module_to_module_mismatch_rate=0.99, wiring_loss_rate=0.99,
            #                 max_power_tracking_rate=1.0, azimuth_in_rad=math.pi / 2.0),
            #     p_values=[], p_min=0.5, p_delta=0.1)
            # null_summary = perform_analysis(P90Client, null_request, call_credentials, show_errors=False)

    except AuthenticationError as e:
        print(f"❌ Authentication failed: {e}")
    except TimeStepDataError as e:
        print(f"❌ Failed to load weather data: {e}")
    except P90ConnectionError as e:
        print(f"❌ Failed to connect to uncertainty service: {e}")
    except (RuntimeError, SystemExit, KeyboardInterrupt) as e:
        print(f"❌ Unexpected error: {e}")

if __name__ == "__main__":
    main()

In [ ]:
# Plotting functions

# Apply default styles to a plotly figure
def apply_styles(fig, image=True):
    fig.update_layout(
        font=dict(size=FIGURE_FONT_SIZE),
        title_font=dict(size=FIGURE_FONT_SIZE+2),
        paper_bgcolor = 'rgba(0,0,0,0)' if TRANSPARENT_PLOTS else 'white',  # Fully transparent background
        plot_bgcolor = 'rgba(0,0,0,0)' if TRANSPARENT_PLOTS else 'white'
    )
    if image:
        # Load the SunSolve Logo and convert to base64
        image_path = 'SunSolveLogo.svg'
        with open(image_path, "rb") as image_file:
            encoded_image = base64.b64encode(image_file.read()).decode()
        fig.update_layout(images=[dict(
            source=f'data:image/svg+xml;base64,{encoded_image}',    # Add SunSolve Logo to plot
            xref="paper", yref="paper",
            x=0.4, y=1.17,
            sizex=0.2, sizey=0.2,
            xanchor="left", yanchor="top"
        )])
    fig.update_xaxes(linewidth=1, linecolor='black', mirror=True, ticks='outside')
    fig.update_yaxes(linewidth=1, linecolor='black', mirror=True, ticks='outside', showgrid=True, gridwidth=GRID_WIDTH, gridcolor=GRID_COLOUR)

def plot_yearly_hist_plotly(data, year_index, y_limit=None, fig_width=900, p_min=0.75, p_delta=0.01):
    fig = go.Figure()
    p_max = 2-p_min-p_delta
    n_p = len(summary.YearlyHistogram[0].bins)
    x_axis = np.linspace(p_min, p_max, n_p)
    tot_freq = sum(data.YearlyHistogram[year_index].bins)
    probs = [freq_i/tot_freq*100 for freq_i in data.YearlyHistogram[year_index].bins]   # normalised frequencies = probabilities
    prob_limit = math.ceil(y_limit/tot_freq*100+0.5) if y_limit is not None else None

    fig.add_trace(go.Bar(x=x_axis, y=probs, marker_color=HISTOGRAM_COLOUR))

    fig.update_layout(
        title=f'Year {data.YearlyHistogram[year_index].Year} Histogram',
        xaxis_title='Relative yield',
        yaxis_title='Probability (%)',
        yaxis_range=[0, prob_limit] if y_limit is not None else None,
        width=fig_width,
    )
    apply_styles(fig)
    fig.update_xaxes(minor_ticks='outside')
    fig.show()

def plot_interactive_histogram(data, p_min=0.75, p_delta=0.01):
    max_freq = max(max(data.YearlyHistogram[i].bins) for i in range(len(data.YearlyHistogram)))
    year_slider = IntSlider(min=1, max=len(data.YearlyHistogram), step=1, description='Year:')

    def update_plot(year_index):
        plot_yearly_hist_plotly(data, year_index-1, y_limit=math.ceil(max_freq/10+0.1)*10, p_min=p_min, p_delta=p_delta)

    interactive_plot = interactive(update_plot, year_index=year_slider)
    display(interactive_plot)

def plot_yearly_P50_Values(data, fig_width=900):
    years, P50Deviations = zip(*[[i.Year, i.Value] for i in data.YearlyP50Deviation])

    fig = go.Figure()
    fig.add_trace(go.Bar(x=list(years), y=list(P50Deviations), marker_color=HISTOGRAM_COLOUR))

    y_range = math.ceil(1000*(max(P50Deviations)-min(P50Deviations)))/1000
    y0 = round(1000*(max(P50Deviations)+min(P50Deviations))/2)/1000

    fig.update_layout(
        title='P50 Yield Relative to Year 1',
        xaxis_title='Year',
        yaxis_title='Relative P50 yield',
        yaxis_range=[y0 - y_range, y0 + y_range],
        width=fig_width,
    )
    apply_styles(fig)
    fig.update_xaxes(dtick=1, minor_ticks=None)
    fig.show()

def plot_pvalues(data):
    # Get unique P values excluding 50
    unique_p_values = sorted(list(set([item.P for item in data.YearlyPValue if item.P != 50])))

    # Separate P values into < 50 and > 50
    p_values_less_than_50 = [p for p in unique_p_values if p < 50]
    p_values_greater_than_50 = [p for p in unique_p_values if p > 50]

    # Create subplots
    if p_values_less_than_50 != []:
        fig = make_subplots(rows=1, cols=2)
        # Add traces for P-values < 50
        for p_value in p_values_less_than_50:
            filtered_data = sorted([item for item in data.YearlyPValue if item.P == p_value], key=lambda x: x.Year)
            years = [item.Year for item in filtered_data]
            p50_deviations = [item.P50Deviation for item in filtered_data]
            fig.add_trace(go.Scatter(x=years, y=p50_deviations, mode='lines', name=f'P{p_value}'), row=1, col=1)

    else:
        fig = make_subplots(rows=1, cols=1)

    # Add traces for P-values > 50
    for p_value in p_values_greater_than_50:
        filtered_data = sorted([item for item in data.YearlyPValue if item.P == p_value], key=lambda x: x.Year)
        years = [item.Year for item in filtered_data]
        p50_deviations = [item.P50Deviation for item in filtered_data]

        if p_values_less_than_50 != []:
            fig.add_trace(go.Scatter(x=years, y=p50_deviations, mode='lines', name=f'P{p_value}'), row=1, col=2)
        else:
            fig.add_trace(go.Scatter(x=years, y=p50_deviations, mode='lines', name=f'P{p_value}'), row=1, col=1)

    # Update layout
    fig.update_layout(
        title=f'P-values{" < 50 and > 50," if p_values_less_than_50 != [] else ""} Relative to Year 1 P50 Yield',
        width=900,
        xaxis={'type': 'category'},
        xaxis2={'type': 'category'},
        showlegend=True,
    )
    apply_styles(fig)
    fig.update_xaxes(title_text='Year', row=1, col=1)
    fig.update_yaxes(title_text='Relative yield', row=1, col=1)
    fig.update_xaxes(title_text='Year', row=1, col=2)

    fig.show()

# Define interactive weather plot function
def interactive_weather_plot(weather_path, weather_data):
    # Get timezone
    if weather_path.lower().endswith('.pvw'):
        with open(weather_path, "rb") as f:
            weather_file_text = f.read()
            weather_setting = weatherData_pb2.WeatherSetting()
            weather_setting.ParseFromString(weather_file_text)
        timezone = datetime.timezone(datetime.timedelta(hours=weather_setting.LocalStandardTimeOffset_Minutes.data/60))
    else:
        timezone = datetime.timezone(datetime.timedelta(hours=0))       # Don't adjust timezone for csv, etc.

    # Extract date and time information and irradiance data
    dates, hours, ghis, dhis, dnis = [], [], [], [], []

    for data_point in weather_data:
        # Convert EndOfPeriodUTC to datetime
        timestamp_seconds = data_point.EndOfPeriodUTC.seconds
        dt_object = datetime.datetime.fromtimestamp(timestamp_seconds, tz=timezone)

        dates.append(dt_object.date())
        hours.append(dt_object.hour)

        # Extract irradiance values, handle cases where they might be missing
        ghis.append(getattr(data_point.Weather, 'GHI', 0))
        dhis.append(getattr(data_point.Weather, 'DHI', 0))
        dnis.append(getattr(data_point.Weather, 'DNI', 0))

    # Prepare max_irradiance variable for keeping plot limits consistent across all days
    max_irradiance = math.ceil(max(ghis + dhis + dnis)/10)*10

    # Create a DataFrame for easier filtering
    weather_df = pd.DataFrame({'Date': dates, 'Hour': hours, 'GHI': ghis, 'DHI': dhis, 'DNI': dnis})

    # Get unique dates
    unique_dates = sorted(weather_df['Date'].unique())

    # Set up interactive plot
    doy_slider = IntSlider(min=1, max=len(unique_dates), step=1, description='Day of year:')

    def update_plot(day_index):
        selected_date = unique_dates[day_index-1]

        # Filter weather_data for the target date and hours 4-20
        daily_data = weather_df[(weather_df['Date'] == selected_date) & (weather_df['Hour'] >= 4) & (weather_df['Hour'] <= 20)]

        # Extract data for plotting
        hours = daily_data['Hour']

        # Create the plotly plot
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=hours, y=daily_data['GHI'], mode='lines', name='GHI', marker_color='#d2226a'))
        fig.add_trace(go.Scatter(x=hours, y=daily_data['DHI'], mode='lines', name='DHI', marker_color='#27cfff'))
        fig.add_trace(go.Scatter(x=hours, y=daily_data['DNI'], mode='lines', name='DNI', marker_color='#febf24'))

        fig.update_layout(
            title=f'Irradiance on {selected_date}',
            xaxis_title='Hour of Day',
            yaxis_title='Irradiance (W/m²)',
            xaxis_range=[4, 20],
            yaxis_range=[0, max_irradiance],
            title_x=0.5,
            height=300,
            width=500,
            margin=dict(l=20, r=20, t=35, b=20),
        )
        apply_styles(fig, image=False)
        fig.update_xaxes(dtick=2)
        fig.show()

    interactive_plot = interactive(update_plot, day_index=doy_slider)
    display(interactive_plot)

In [ ]:
# Define distribution functions for plotting interactively
def gaussian(x, x0, std_dev):
    """Calculates the probability density function of a Gaussian distribution."""
    # Ensure x is a numpy array for correct operation with scipy.stats
    x = np.asarray(x)
    return norm.pdf(x, loc=x0, scale=std_dev)

def weibull(x, x0, k, lamb, positive_polarity=False):
    """Calculates the probability density function of a Weibull distribution."""
    p = 1 if positive_polarity else -1
    # 0 if p * (x - x0) < 0
    return 0 if p*(x-x0)<0 else (k/lamb)*(p*(x-x0)/lamb)**(k-1)*math.exp(-1*(p*(x-x0)/lamb)**k)

def skewed_gaussian(x, x0, std_dev, skewness):
    """Calculates the probability density function of a Skewed Gaussian distribution."""
    # ζ = location (x0), ω = scale (std_dev), α = skewness
    x = np.asarray(x)    # Ensure x is a numpy array for correct operation with scipy.stats
    return skewnorm.pdf(x, skewness, loc=x0, scale=std_dev)

def delta(x, x0, height=1.0):
    """Represents a Delta function."""
    # For plotting, we'll represent this as a single point
    return height if x == x0 else 0

def tophat(x, x0, x1, height):
    """Represents a Tophat (uniform) distribution."""
    return height if x0 <= x <= x1 else 0

def constant(x, value):
    """Represents a Constant distribution."""
    return value

# Define parameters for each distribution type
distribution_params = {
    'Gaussian': ['x0', 'std_dev'],
    'Skewed Gaussian': ['x0', 'std_dev', 'skewness'],
    'Delta': ['x0', 'height'],
    'Tophat': ['x0', 'x1', 'height'],
    'Constant': ['value'],
    'Weibull': ['x0', 'k', 'lamb']
}

# Define interactive distribution plotter function, to visualise possible modifiers
def interactive_distribution_plot():
    # Create specific input widgets for each distribution type (initially hidden), also setting default/starting values
    std_dev_input = widgets.FloatText(value=0.05, description='σ:', layout=widgets.Layout(width='70%', display='none'), step=0.01)
    x0_input = widgets.FloatText(value=1.0, description='x0:', layout=widgets.Layout(width='70%', display='none'), step=0.01)
    k_input = widgets.FloatText(value=2.0, description='k:', layout=widgets.Layout(width='70%', display='none'), step=0.1)
    lamb_input = widgets.FloatText(value=0.05, description='λ:', layout=widgets.Layout(width='70%', display='none'), step=0.01)
    skewness_input = widgets.FloatText(value=2.0, description='skewness:', layout=widgets.Layout(width='70%', display='none'), step=0.1)
    height_input = widgets.FloatText(value=1.0, description='height:', layout=widgets.Layout(width='70%', display='none'), step=0.1)
    value_input = widgets.FloatText(value=1.0, description='value:', layout=widgets.Layout(width='70%', display='none'), step=0.1)
    x1_input = widgets.FloatText(value=1.2, description='x1:', layout=widgets.Layout(width='70%', display='none'), step=0.1)

    # Store all parameter widgets in a dictionary for easy access
    param_widgets = {
        'std_dev': std_dev_input,
        'x0': x0_input,
        'x1': x1_input,
        'k': k_input,
        'lamb': lamb_input,
        'skewness': skewness_input,
        'height': height_input,
        'value': value_input,
        'x_min': None, # These will be handled by the main xmin_input and xmax_input
        'x_max': None
    }

    # Create main control widgets
    distribution_dropdown = widgets.Dropdown(
        # options=['Gaussian', 'Skewed Gaussian', 'Weibull', 'Delta', 'Tophat', 'Constant'],
        options=['Gaussian', 'Skewed Gaussian', 'Weibull'],
        value='Gaussian',
        description='Distribution:',
        layout=widgets.Layout(width='30%')
    )

    plot_header = widgets.HTML(
        value='<b>Plot controls:</b>',
        description='',
    )
    xmin_input = widgets.FloatText(value=0.0, description='x_min:', layout=widgets.Layout(width='17%'), step=0.1)
    xmax_input = widgets.FloatText(value=2.0, description='x_max:', layout=widgets.Layout(width='17%'), step=0.1)
    n_input = widgets.IntText(value=200, description='n_points:', layout=widgets.Layout(width='17%'), step=50)

    # Create an Output widget to specifically hold the plot output
    plot_output = widgets.Output()

    def update_param_visibility(distribution_type):
        """Updates the visibility of parameter widgets based on the selected distribution type."""
        # Hide all parameter widgets first
        for param_widget in [std_dev_input, x0_input, x1_input, k_input, lamb_input, skewness_input, height_input, value_input]:
            param_widget.layout.display = 'none'

        # Show relevant widgets based on the selected distribution type
        if distribution_type in distribution_params:
            for param_name in distribution_params[distribution_type]:
                if param_name in param_widgets and param_widgets[param_name] is not None:
                    param_widgets[param_name].layout.display = ''


    def plot_distribution(distribution_type, xmin, xmax, n, **params):
        """Plots the selected distribution function."""
        distributions = {
            'Gaussian': gaussian,
            'Weibull': weibull,
            'Skewed Gaussian': skewed_gaussian,
            'Delta': delta,
            'Tophat': tophat,
            'Constant': constant
        }

        if distribution_type not in distributions:
            print("Invalid distribution type selected.")
            return

        x = np.linspace(xmin, xmax, n+1)
        y = []
        arguments = []  # parameters for the user to use to produce this distribution

        # Use the Output widget as the context for clearing and displaying the plot
        with plot_output:
            clear_output(wait=True)
            x0 = params.get('x0', (xmin + xmax) / 2)    # Default to (xmin+xmax)/2 if missing (shouldn't be)

            # Handle Delta function separately for plotting
            if distribution_type == 'Delta':
                height = params.get('height', 1.0)
                # For delta, we represent it as a single point at x0 with a stem plot
                print(f"\t\t['Delta', {', '.join(f'{i:g}' for i in arguments)}]")     # Show user the command to define this error function
                fig = go.Figure()
                fig.add_trace(go.Scatter(x=[x0], y=[height], mode='markers', name='Relative frequency', marker_color=HISTOGRAM_COLOUR))
                fig.update_layout(xaxis_title='x', yaxis_title='Relative frequency', width=400, height=350, margin=dict(l=20, r=20, t=15, b=20))
                apply_styles(fig, image=False)
                fig.update_yaxes(showgrid=True, gridwidth=GRID_WIDTH, gridcolor=GRID_COLOUR)
                fig.show()
                arguments = [x0, height]
            else:
                dist_func = distributions[distribution_type]
                if distribution_type == 'Weibull':
                    # Ensure k and lamb are present in params, use defaults if not
                    k_val = params.get('k', 5)
                    lamb_val = params.get('lamb', 3)
                    y = [dist_func(xi, x0, k_val, lamb_val) for xi in x]
                    arguments = [x0, k_val, lamb_val]
                elif distribution_type == 'Constant':
                    # Ensure value is present in params, use default if not
                    value_val = params.get('value', 0.0)
                    y = [dist_func(xi, value_val) for xi in x]
                    arguments = [value_val]
                elif distribution_type == 'Tophat':
                    # Ensure height is present in params, use default if not
                    height_val = params.get('height', 1.0)
                    x1 = params.get('x1', 3.0)
                    y = [dist_func(xi, x0, x1, height_val) for xi in x]
                    arguments = [x0, x1, height_val]
                elif distribution_type == 'Skewed Gaussian':
                    std_dev = params.get('std_dev', 0.1)
                    skewness = params.get('skewness', 1.0)
                    y = [dist_func(xi, x0, std_dev, skewness) for xi in x]
                    arguments = [skewness, x0, std_dev]
                else: # Gaussian
                    std_dev = params.get('std_dev', 0.1)
                    y = [dist_func(xi, x0, std_dev) for xi in x]
                    arguments = [x0, std_dev]

                # Show user the command to define this distribution (e.g. "['Gaussian', 1, 0.05]")
                print(f"\t\t['{distribution_type.replace(' ', '')}', {', '.join(f'{i:g}' for i in arguments)}]")
                # Normalise the y-values to have 1 as the maximum
                y = [yi/max(y) for yi in y]
                # Plot the distribution
                fig = go.Figure()
                fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='Relative frequency', marker_color=HISTOGRAM_COLOUR))
                fig.update_layout(xaxis_title='x', yaxis_title='Relative frequency', width=400, height=350, margin=dict(l=20, r=20, t=15, b=20))
                apply_styles(fig, image=False)
                fig.show()


    # Link the widgets to the interactive wrapper function
    def interactive_plot_wrapper(distribution_type, xmin, xmax, n, **param_values):
        """Wrapper function to handle interactive updates and call plot_distribution."""
        update_param_visibility(distribution_type)

        # Prepare parameters for plot_distribution based on the selected type
        params_to_pass = {}
        if distribution_type in distribution_params:
            for param_name in distribution_params[distribution_type]:
                # Check if the parameter is one of the specific distribution parameters and exists in param_values
                if param_name in ['mean', 'std_dev', 'x0', 'x1', 'k', 'lamb', 'skewness', 'height', 'value'] and param_name in param_values:
                    params_to_pass[param_name] = param_values[param_name]
                # x_min and x_max for Tophat will be taken from the main xmin and xmax inputs in plot_distribution

        plot_distribution(distribution_type, xmin, xmax, n, **params_to_pass)


    interactive_plot = widgets.interactive(
        interactive_plot_wrapper,
        distribution_type=distribution_dropdown,
        xmin=xmin_input,
        xmax=xmax_input,
        n=n_input,
        # Pass all potential parameter widgets to interactive, their values will be available in param_values
        std_dev=param_widgets['std_dev'],
        x0=param_widgets['x0'],
        x1=param_widgets['x1'],
        k=param_widgets['k'],
        lamb=param_widgets['lamb'],
        skewness=param_widgets['skewness'],
        height=param_widgets['height'],
        value=param_widgets['value']
    )

    # Group the parameter specific widgets in two columns
    param_widget_list = [x0_input, x1_input, std_dev_input, skewness_input, k_input, lamb_input, height_input, value_input]
    # Split the widgets into two columns (adjust as needed based on the number of parameters)
    col1_widgets = param_widget_list[::2] # Every other widget starting from the first
    col2_widgets = param_widget_list[1::2] # Every other widget starting from the second

    # Group the main control widgets
    control_widgets = widgets.VBox([
        distribution_dropdown,
        widgets.HBox([widgets.VBox(col1_widgets), widgets.VBox(col2_widgets)])
    ])
    # Group the plot control widgets
    plot_widgets = widgets.VBox([
        plot_header,
        widgets.HBox([xmin_input, xmax_input, n_input], layout=widgets.Layout(width='95%')), # Arrange xmin and xmax horizontally
    ])


    # Combine control widgets, parameter widgets, and the interactive plot output
    interactive_display = widgets.VBox([
        # widgets.Label("Interactive Distribution Modifier"), # Add a title
        control_widgets,
        plot_output, # Display the dedicated Output widget for the plot
        plot_widgets,
    ])

    # Display the organized interactive plot
    display(interactive_display)

    # Initially update parameter visibility based on the default dropdown value
    update_param_visibility(distribution_dropdown.value)

    # Trigger an initial plot display
    interactive_plot_wrapper(distribution_dropdown.value, xmin_input.value, xmax_input.value, n_input.value,
                            std_dev=std_dev_input.value, x0=x0_input.value, x1=x1_input.value, k=k_input.value,
                            lamb=lamb_input.value, skewness=skewness_input.value,
                            height=height_input.value, value=value_input.value)

In [ ]:
# Define export function
def export_to_excel(data, filename, p_min=0.75, p_delta=0.01):
    # Bins go: var numBins = (int)((1 - request.MinP) * 2 / request.DeltaP + 0.5);

    # Process YearlyPValue data
    pvalues_data = {}
    unique_p_values = sorted(list(set([item.P for item in data.YearlyPValue])))
    unique_years = sorted(list(set([item.Year for item in data.YearlyPValue])))

    pvalues_data['PValue'] = unique_p_values
    for year in unique_years:
        year_data = [item.P50Deviation for item in data.YearlyPValue if item.Year == year]
        pvalues_data[f'Year {year}'] = year_data

    pvalues_df = pd.DataFrame(pvalues_data)

    # Process YearlyP50Deviation data
    yearly_p50_data = {}
    p50_values = [item.Value for item in summary.YearlyP50Deviation]
    yearly_p50_data['P50Deviation'] = ['P50']
    for year, p50 in zip(unique_years, p50_values):
        yearly_p50_data[f'Year {year}'] = p50

    p50_df = pd.DataFrame(yearly_p50_data)

    # Process YearlyHistogram data
    histograms_data = {}
    unique_years_hist = sorted(list(set([item.Year for item in data.YearlyHistogram])))

    # Define histogram bins based on p_min and p_delta
    num_bins = len(data.YearlyHistogram[0].bins)
    p_max = 1 + (1-p_min) - p_delta
    # bin_labels = [f'{i:.2f}' for i in np.linspace(p_min, p_max, num_bins)]    # Have them as strings so first can be '<p_min' (abandoned)
    bin_labels = [round(i, 2) for i in np.linspace(p_min, p_max, num_bins)]

    # # Add custom labels for the ends as requested, assuming they correspond to the first and last bins
    # bin_labels[0] = f'<{p_min:.2f}'     # The first bin is less than p_min
    # bin_labels[-1] = f'>{p_max:.2f}'    # The last bin is greater than p_max+p_delta

    histograms_data['Bin'] = bin_labels

    for year in unique_years_hist:
        year_histogram = [list(item.bins) for item in data.YearlyHistogram if item.Year == year][0]
        histograms_data[f'Year {year}'] = year_histogram

    histograms_df = pd.DataFrame(histograms_data)

    # Save to Excel
    if filename.endswith('.xlsx'):
        filename = filename[:-5]
    with pd.ExcelWriter(f'{filename}.xlsx') as writer:
        pvalues_df.to_excel(writer, sheet_name='PValues', index=False)
        p50_df.to_excel(writer, sheet_name='P50Deviations', index=False)
        histograms_df.to_excel(writer, sheet_name='Histograms', index=False)

    print(f"Summary data saved to '{filename}.xlsx'")